In [2]:
import numpy as np
from szar.counts import ClusterCosmology,Halo_MF
from szar.szproperties import SZ_Cluster_Model
from configparser import SafeConfigParser
from orphics.tools.io import dictFromSection,listFromConfig
import cPickle as pickle

home = '/Users/nab/Repos/SZ_filter/'

!pwd

iniFile = home+'input/pipeline.ini'
expName = 'CCATP-propv2'
gridName = 'grid-owl2'
cal = 'owl2'

Config = SafeConfigParser()
Config.optionxform=str
Config.read(iniFile)

fparams = {}
for (key, val) in Config.items('params'):
    if ',' in val:
        param, step = val.split(',')
        fparams[key] = float(param)
    else:
        fparams[key] = float(val)

bigDataDir = Config.get('general','bigDataDirectory')
clttfile = Config.get('general','clttfile')
constDict = dictFromSection(Config,'constants')
clusterDict = dictFromSection(Config,'cluster_params')
beam = listFromConfig(Config,expName,'beams')
noise = listFromConfig(Config,expName,'noises')
freq = listFromConfig(Config,expName,'freqs')
lknee = listFromConfig(Config,expName,'lknee')[0]
alpha = listFromConfig(Config,expName,'alpha')[0]

calFile = bigDataDir+"lensgrid_grid-"+cal+"_"+cal+".pkl"

version = Config.get('general','version')

mexp_edges, z_edges, lndM = pickle.load(open(calFile,"rb"))

mgrid,zgrid,siggrid = pickle.load(open(bigDataDir+"szgrid_"+expName+"_"+gridName+ "_v" + version+".pkl",'rb'))

assert np.all(mgrid==mexp_edges)
assert np.all(z_edges==zgrid)

cc = ClusterCosmology(fparams,constDict,clTTFixFile=clttfile)
HMF = Halo_MF(cc,mgrid,zgrid)

qs = listFromConfig(Config,'general','qbins')
qspacing = Config.get('general','qbins_spacing')
if qspacing=="log":
    qbin_edges = np.logspace(np.log10(qs[0]),np.log10(qs[1]),int(qs[2])+1)
elif qspacing=="linear":
    qbin_edges = np.linspace(qs[0],qs[1],int(qs[2])+1)
else:
    raise ValueError
    
SZProp = SZ_Cluster_Model(cc,clusterDict,rms_noises = noise,fwhms=beam,freqs=freq,lknee=lknee,alpha=alpha)

Mwl = 10**HMF.Mexp
z_arr = HMF.zarr
M_arr =  np.outer(HMF.M,np.ones([len(z_arr)]))

#if HMF.sigN is None: HMF.updateSigN(SZProp)
#sigN = HMF.sigN
#np.savetxt(home+'/tests/sigN_arr_save.txt',sigN)

sigN = np.loadtxt(home+'/tests/sigN_arr_save.txt')

#SZProp = SZ_Cluster_Model(cc,clusterDict,rms_noises = noise,fwhms=beam,freqs=freq,lknee=lknee,alpha=alpha)

q_arr = (qbin_edges[1:]+qbin_edges[:-1])/2.

i = 5
kk = 2
jj = 4

blah_orig = SZProp.P_of_qn(SZProp.lnY,M_arr[:,i],z_arr[i],sigN[:,i],q_arr[kk])
blah = SZProp.P_of_qn_corr(SZProp.lnY,M_arr[:,i],z_arr[i],sigN[:,i],q_arr[kk],Mwl[jj],lndM[:,i])

print blah_orig
print blah

#dN_dmqz_corr = HMF.N_of_mqz_SZ_corr(lndM,qbin_edges,SZProp)                                                                  
#dN_dmqz = HMF.N_of_mqz_SZ(lndM,qbin_edges,SZProp)

/Users/nab/Repos/SZ_filter
((222, 130), (), (222,))
No corr
((222, 130), (222, 130), (130,), (129,))
size 2
((222, 130), (), (222,), (222,))
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.]
(222, 130)
(2, 222, 130)
(222, 130)
corr
((222, 130), (222, 130), (130,), (129,))
[  9.53129162e-008   1.04742799e-007   1.15462972e-007   1.27686692e-007
   1.41667977e-007   1.57710369e-007   1.76177786e-007   1.97507995e-007
   2.22229397e-007   2.50982015e-007   2.84543816e-007   3.23863856e-007
   3.70104138e-007   4.246926

   1.16483167e+35   1.21796870e+35]


/Users/nab/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:17: DeprecationWarning: The SafeConfigParser class has been renamed to ConfigParser in Python 3.2. This alias will be removed in future versions. Use ConfigParser directly instead.
/Users/nab/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:19: DeprecationWarning: You passed a bytestring as `filenames`. This will not work on Python 3. Use `cp.read_file()` or switch to using Unicode strings across the board.
